# Local Sensors - Getting data

### Assembly the Hw

<p><img src="https://github.com/Mjrovai/Python4DS/blob/master/Micropython/IoT_TS_MQTT/electrDiagr.png?raw=true"> </p>



### Connect with ESP8266 via serial port

In [1]:
%serialconnect to --port=/dev/tty.SLAB_USBtoUART --baud=115200

Connecting to --port=/dev/tty.SLAB_USBtoUART --baud=115200 
Ready.


### DHT Sensor

In [2]:
from dht import DHT22
from machine import Pin
dht22 = DHT22(Pin(12))

In [3]:
# Function to read DHT
def readDht():
    dht22.measure()
    return dht22.temperature(), dht22.humidity()

In [4]:
# test DHT function
print (readDht())

(18.6, 41.8)


### DS18B20 sensor

In [5]:
# Import libraries 
import onewire, ds18x20
import time

In [6]:
# Define which pin the 1-wire device will be connected ==> pin 2 (D4)
dat = Pin(2)

In [7]:
# create the onewire object
ds = ds18x20.DS18X20(onewire.OneWire(dat))

In [8]:
# scan for devices on the bus
sensors = ds.scan()
print('found devices:', sensors)

found devices: [bytearray(b'(\xff\xd3\xe6a\x16\x041')]


In [9]:
# function to read DS18B20 
def readDs(): 
    ds.convert_temp()
    time.sleep_ms(750)
    return round(ds.read_temp(sensors[0]), 1)

In [10]:
# test readDs function
print(readDs())

18.9


### LDR Sensor

In [11]:
# import library
from machine import ADC

In [12]:
# Define object
adc = ADC(0)

In [13]:
print(adc.read())

821


In [14]:
#function to read luminosity
def readLdr():
    lumPerct = (adc.read()-40)*(10/86) # convert in percentage ("map")
    return round(lumPerct)

In [15]:
# test LDR read function
print (readLdr())

91


### Button (Digital Input)

In [16]:
# define pin 13 as an input and activate an internal Pull-up resistor:
button = Pin(13, Pin.IN, Pin.PULL_UP)

In [17]:
# Function to read button state:
def readBut():
        return button.value()

In [18]:
# test button read function
print (readBut())

1


### Define Function to collect sensor data

In [19]:
def colectData():
    temp, hum, = readDht()
    extTemp = readDs()
    lum = readLdr()
    butSts = readBut()
    return temp, hum, extTemp, lum, butSts

In [20]:
# test function
print(colectData())

(18.7, 41.6, 18.9, 92, 1)


### Displaying Data on OLED

In [21]:
# import library and create object i2c
from machine import I2C
i2c = I2C(scl=Pin(5), sda=Pin(4))

In [22]:
# import library and create object oled
import ssd1306
i2c = I2C(scl=Pin(5), sda=Pin(4))
oled = ssd1306.SSD1306_I2C(128, 64, i2c, 0x3c)

In [23]:
# test OLED
oled.fill(0)
oled.text("Hello esp8266", 0, 0)
oled.show()

In [24]:
# Read Sensors
temp, hum, extTemp, lum, butSts = colectData()

In [25]:
# Show data on display once 
oled.fill(0)
oled.text("Temp:    " + str(temp) + "oC", 0, 4)
oled.text("Hum:     " + str(hum) + "%",0, 16)
oled.text("ExtTemp: " + str(extTemp) + "oC", 0, 29)
oled.text("Lumin:   " + str(lum) + "%", 0, 43)
oled.text("Button:  " + str(butSts), 0, 57)
oled.show()

In [26]:
# create a function to display data:
def displayData(temp, hum, extTemp, lum, butSts):
    oled.fill(0)
    oled.text("Temp:    " + str(temp) + "oC", 0, 4)
    oled.text("Hum:     " + str(hum) + "%",0, 16)
    oled.text("ExtTemp: " + str(extTemp) + "oC", 0, 29)
    oled.text("Lumin:   " + str(lum) + "%", 0, 43)
    oled.text("Button:  " + str(butSts), 0, 57)
    oled.show()

In [27]:
# define pin 0 as output
led = Pin(0, Pin.OUT)

In [28]:
# display data with a function
led.on()
temp, hum, extTemp, lum, butSts = colectData()
displayData(temp, hum, extTemp, lum, butSts)
led.off()

In [29]:
# Clear display :
def displayClear():
    oled.fill(0)
    oled.show()

In [30]:
from time import sleep

In [31]:
# create a blink function
def blinkLed(num):
    for i in range(0, num):
        led.on()
        sleep(0.5)
        led.off()
        sleep(0.5)

In [32]:
PUB_TIME_SEC = 30

In [33]:
# loop getting data until button is pressed
while button.value():
        led.on()
        temp, hum, extTemp, lum, butSts = colectData()
        displayData(temp, hum, extTemp, lum, butSts)
        led.off()
        time.sleep(PUB_TIME_SEC)
blinkLed(3)
displayClear()

.............